In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
df = pd.read_pickle('accidentes.pkl')

In [ ]:
df['dia_semana'] = df['fecha'].dt.dayofweek
df['dia_mes'] = df['fecha'].dt.day
df['mes'] = df['fecha'].dt.month
df['año'] = df['fecha'].dt.year

In [ ]:
df['rango_horario'] = pd.to_timedelta(df['rango_horario'])
df['hora'] = df['rango_horario'].dt.components['hours']
df.loc[df['hora'] == 0, 'hora'] = 24

- **accidente_id:** Identificador único para cada accidente.
- **fecha_completa:** Fecha y hora completa del accidente.
- **fecha:** Fecha del accidente.
- **rango_horario:** Fango horario en el que ocurrió el accidente.
- **localizacion:** Ubicación específica del accidente.
- **numero:** Número asociado con la ubicación del accidente.
- **distrito:** Distrito donde ocurrió el accidente.
- **coordenada_x:** Coordenada X de la ubicación del accidente.
- **coordenada_y:** Coordenada Y de la ubicación del accidente.
- **condicion:** Condiciones en las que ocurrió el accidente (favorable o desfavorable).
- **lesividad:** Nivel de lesión resultante del accidente.
- **persona_implicada:** Persona implicada en el accidente (conductor, pasajero, etc.).
- **positiva_alcohol:** Si la persona implicada dio positivo por alcohol.
- **positiva_droga:** Si la persona implicada dio positivo por drogas.
- **rango_edad:** Rango de edad de la persona implicada.
- **sexo:** Sexo de la persona implicada.
- **tipo_accidente:** Tipo de accidente (colisión doble, colisión múltiple, etc.).
- **tipo_vehiculo:** Tipo de vehículo involucrado en el accidente.
- **victimas:** Número de víctimas involucradas en el accidente.

In [ ]:
positivo_alcohol_edad = df[df["positiva_alcohol"] == 1].groupby(["rango_edad"]).size().reset_index(name="numero_positivos")
positivo_alcohol_edad = positivo_alcohol_edad.sort_values(by='numero_positivos', ascending=False)
positivo_alcohol_edad

In [ ]:
fig = px.bar(data_frame     = positivo_alcohol_edad,
             x              = "rango_edad",
             y              = "numero_positivos",
             opacity        = 0.5,
             title          = "Nº de positivos en alcohol por rangos de edad",
             height         = 700,
             color_discrete_sequence=['olivedrab']
            )
fig.update_layout(xaxis={'title'   : 'Rangos',
                         'tickmode': 'array', 
                         'tickvals': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
                         'ticktext': ["15-17", "18-20", "21-24", "25-29", "30-34", "35-39", "40-44", "45-49", "50-54", "55-59", "60-64", "65-69", "70-74", "+74"]},
                  title_x=0.5)

fig.show()

In [ ]:
# Cuando empieza a haber registros de alcohol y drogas, a contemplarse esa variable 
df_drogas_alcohol= df[["accidente_id", "año", "positiva_droga", "positiva_alcohol", "dia_semana", "rango_horario", "sexo"]]
df_drogas_alcohol = df_drogas_alcohol[df_drogas_alcohol['año'] >= 2019]

df_drogas_alcohol

In [ ]:
distritos = df['distrito'].unique()

distritos

In [ ]:
distritos = df.groupby([df['distrito'], df['fecha'].dt.year])['accidente_id'].nunique().reset_index(name='count')

# Bucle para visualizar la accidentalidad por año y distrito
for distrito in df['distrito'].unique()[:-1]:
    distrito_data = distritos[distritos['distrito'] == distrito].sort_values(by='count', ascending=False)
    plt.figure(figsize=(10, 6))
    sns.barplot(y='fecha', x='count', data=distrito_data, orient='h', order=distrito_data.sort_values('count', ascending=False)['fecha'], palette='viridis')
    plt.title(f'Cantidad de accidentes por año en el distrito {distrito.capitalize()}')
    plt.xlabel('Año')
    plt.ylabel('Cantidad de accidentes')
    plt.show()

In [ ]:
condiciones = df.groupby('condicion')['accidente_id'].nunique().reset_index().sort_values(by= 'accidente_id', ascending = False)

plt.figure(figsize=(12, 6))
sns.barplot(x = 'accidente_id', y = 'condicion', data = condiciones, hue = 'condicion', palette = 'viridis')

plt.title('Cantidad de accidentes por condiciones')
plt.xlabel('condiciones')
plt.ylabel('Cantidad de accidentes')
plt.tight_layout()

plt.show()

In [ ]:
grupo_persona_sexo = df.groupby(['persona_implicada', 'sexo'])['accidente_id'].nunique().reset_index(name = 'count')
plt.figure(figsize = (10, 6))
sns.barplot(data = grupo_persona_sexo, x ='persona_implicada', y='count', hue = 'sexo')
plt.title('Distribución de género por persona implicada en accidentes')
plt.xlabel('Persona Implicada')
plt.ylabel('Número de accidentes')
plt.legend(title='Género')
plt.show()

In [ ]:
lesividad = df.groupby('lesividad')['accidente_id'].nunique().reset_index()

plt.figure(figsize=(15, 6))
ax = sns.barplot(y=lesividad['lesividad'], x=lesividad['accidente_id'], color='red')

# Agregar el número en cada barra
for index, value in enumerate(lesividad['accidente_id']):
    ax.text(value, index, str(value), color='black', ha="left")

plt.title('Número de accidentes por lesividad')
plt.xlabel('Número de accidentes')
plt.ylabel('Tipos de lesiones')
plt.tight_layout()
plt.show()

In [ ]:
diccionario_edades = {
         1 : 'menor de 5 años',
         1 : 'de 0 a 5 años',
         2 : 'de 06 a 9 años',
         3 : 'de 10 a 14 años',
         4 : 'de 15 a 17 años',
         5 : 'de 18 a 20 años',
         6 : 'de 21 a 24 años',
         7 : 'de 25 a 29 años',
         8 : 'de 30 a 34 años',
         8 : 'de 30 a 34 anos',
         9 : 'de 35 a 39 años',
        10 : 'de 40 a 44 años',
        11 : 'de 45 a 49 años',
        12 : 'de 50 a 54 años',
        13 : 'de 55 a 59 años',
        14 : 'de 60 a 64 años',
        15 : 'de 65 a 69 años',
        16 : 'de 70 a 74 años',
        17 : 'mas de 74 años',
        17 : 'de mas de 74 años ',
        17 : 'de más de 74 años',
    np.nan : 'desconocido',
    np.nan : 'desconocida'
}

df['rango_edad'] = df['rango_edad'].map(diccionario_edades)
accidentes_por_edad = df.groupby('rango_edad')['accidente_id'].nunique()
# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
accidentes_por_edad.plot(kind='barh')
plt.title('Cantidad de Accidentes por Rango de Edad')
plt.xlabel('Rango de Edad')
plt.ylabel('Cantidad de Accidentes')

plt.show()

In [ ]:
accidentes_por_edad_sexo = df.groupby(['rango_edad', 'sexo'])['accidente_id'].count().unstack()
plt.figure(figsize = (10, 6))
accidentes_por_edad_sexo.plot(kind = 'barh', color = ['skyblue', 'pink'], figsize = (10,6))
plt.title('Cantidad de Accidentes por Rango de Edad y Sexo')
plt.xlabel('Cantidad de Accidentes')
plt.ylabel('Rango de Edad')
plt.grid(axis = 'x', linestyle = '--', alpha = 0.7)
plt.tight_layout()

In [ ]:
df.groupby(["fecha", "accidente_id", "distrito"]).size().reset_index(name='num_accidentes')

# Conclusiones

Más allá del número de accidentes, los datos no aportan unos datos significativos para el estudio de predicciones de accidentes.
A simple vista podemos ver que sería en Octubre el mes con mayor frecuencia de accidentes, junto con los viernes, y las horas puntas.

Por otro lado también se observa una accidentalidad meno en distritos de sesgo soscial menor, lo que deja a entender queno se notifica adecuadamente los partes.

Con respecto a la edad de los involucrados en los accidentes, predominan los que entran dentro del rango de edad para la conducción, sobretodo en varones.